In [0]:
# library
import sys
import pandas as pd
import numpy as np

## Load dataset

In [0]:
# Google Drive drivers 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# load data from google drive
downloaded = drive.CreateFile({'id':'xxx'}) # copy the shared link, past the id here
downloaded.GetContentFile('train.csv')  #file name
df = pd.read_csv('train.csv')

In [0]:
df.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


## Get abnormal dataset (drop rows with all 0 for label)

In [0]:
mask1 = df['toxic'] != 0
mask2 = df['severe_toxic'] !=0
mask3 = df['obscene'] !=0
mask4 = df['threat'] !=0
mask5 = df['insult'] !=0
mask6 = df['identity_hate'] !=0

In [0]:
df_abnormal = df[mask1 | mask2 |mask3 | mask4 | mask5 |mask6]

In [0]:
df_abnormal.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
12,0005c987bdfc9d4b,Hey... what is it..\n@ | talk .\nWhat is it......,1,0,0,0,0,0
16,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming ...",1,0,0,0,0,0
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0


In [0]:
df_abnormal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16225 entries, 6 to 159554
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             16225 non-null  object
 1   comment_text   16225 non-null  object
 2   toxic          16225 non-null  int64 
 3   severe_toxic   16225 non-null  int64 
 4   obscene        16225 non-null  int64 
 5   threat         16225 non-null  int64 
 6   insult         16225 non-null  int64 
 7   identity_hate  16225 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 1.1+ MB


## Split data by index

In [0]:
# If use the abnormal dataset to do augmentation without split, it will taks around 6 hours.
# split dataset to 4 sub datasets to save time, each subset takes around 90 minites to get the augmentation result
df_copy = df_abnormal[4050:8100] # split2

In [0]:
df_copy.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
38940,67e084702b37484c,who has stepped in from nowhere and is interfe...,1,0,0,0,0,0
38956,67eb3dd3398f7c8f,I still dont know what the hell I did Assholes!,1,0,1,0,0,0
38964,67f03d5ff349eb91,"""\n\n I bet you lick a lot of cock \n\nHave yo...",1,0,1,0,1,0
38972,67f3fdb8302f31aa,Transsexual \n\nI saw you added a transsexual ...,1,0,0,0,0,0
38987,67fde260ec36809d,"""\n\n BREAKING NEWS!!!! I KNOW THIS IS ONLY F...",1,0,0,0,0,0


In [0]:
df_copy.tail()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
78687,d28877e35690949d,A silly fat cow who won't leave me alone,1,0,1,0,1,0
78704,d2917a374a7aff74,fuck you\n\nfuck you majorly. suck my dick ass...,1,1,1,0,1,0
78711,d29436ff5319353b,I DETEST you \n\nI absolutely hate you. You ar...,1,0,1,0,1,0
78716,d2984c0f25a548c1,JLaTondre is a gay boy who likes to suck dick ...,1,0,1,0,1,1
78727,d29ea4ec66b1b882,And why the fuck should i care? \n\n67.165.71.214,1,0,1,0,0,0


In [0]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4050 entries, 38940 to 78727
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             4050 non-null   object
 1   comment_text   4050 non-null   object
 2   toxic          4050 non-null   int64 
 3   severe_toxic   4050 non-null   int64 
 4   obscene        4050 non-null   int64 
 5   threat         4050 non-null   int64 
 6   insult         4050 non-null   int64 
 7   identity_hate  4050 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 284.8+ KB


## Text data augmentation
To extend our training dataset, so as to achieve better model classification capacity for deep recurrent neural networks. We translate a comment to to an intermediate language (e.g. French, Chinese) and then translate it back to English.

In [0]:
from textblob import TextBlob
from textblob.translate import Translator
from textblob.exceptions import TranslatorError, NotTranslated
import time

In [0]:
# use time.sleep, time range tested from 0.2 with step of 0.05, and 0.5 is best choice
def translate(comment, language):
    if hasattr(comment, 'decode'):
        comment = comment.decode('utf-8')

    text = TextBlob(comment)
    try:
        text = text.translate(from_lang='en', to=language)
        time.sleep(0.5)
        text = text.translate(from_lang=language, to='en')
        time.sleep(0.5)
    except NotTranslated: 
      pass

    return str(text)

### translate splitted data - comment

In [0]:
translate_text = df_copy['comment_text']

In [0]:
translate_fr = [translate(entry, 'fr') for entry in translate_text] # use 'zh-CN' for case: transfer to chineses then transfer back to english

In [0]:
len(translate_fr)

4050

In [0]:
#create bew df

df_new = pd.DataFrame({'text_fr':translate_fr})

In [0]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4050 entries, 0 to 4049
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   text_fr  4050 non-null   object
dtypes: object(1)
memory usage: 31.8+ KB


In [0]:
#add new column text_fr
#df_copy['text_fr'] = translate_fr

In [2]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [0]:
df_new.to_csv('fr_split2.csv')
!cp fr_split2.csv "drive/My Drive/"

### Combine splitted augmentation result if you split dataset as above
1. combined the augmentation result in the same sequence as the split sequence;
2. add the combined result to abnormal dataset as new column, named 'text_fr' and 'text_cn'

### check the final dataset after augmentation:

In [0]:
# data augmentation - fr
df_aug_fr = pd.read_csv('data_augmentation_fr.csv')

In [6]:
df_aug_fr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16225 entries, 0 to 16224
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     16225 non-null  int64 
 1   id             16225 non-null  object
 2   comment_text   16225 non-null  object
 3   text_fr        16225 non-null  object
 4   toxic          16225 non-null  int64 
 5   severe_toxic   16225 non-null  int64 
 6   obscene        16225 non-null  int64 
 7   threat         16225 non-null  int64 
 8   insult         16225 non-null  int64 
 9   identity_hate  16225 non-null  int64 
dtypes: int64(7), object(3)
memory usage: 1.2+ MB


In [9]:
df_aug_fr.head()

,Unnamed: 0,id,comment_text,text_fr,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,COCKSUCKER BEFORE PISSING AROUND MY WORK,1,1,1,0,1,0
1,1,0005c987bdfc9d4b,Hey... what is it..\n@ | talk .\nWhat is it......,Hey ... what is it ..\n@ | speak .\nWhat is it...,1,0,0,0,0,0
2,2,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming ...","Goodbye!\n\nDon't look, don't come and don't t...",1,0,0,0,0,0
3,3,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,Are you gay or anti-Semitic?\n\nArchangel Whit...,1,0,1,0,1,1
4,4,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!","FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0


In [0]:
# data augmentation - cn
df_aug_cn = pd.read_csv('data_augmentation_cn_v2.csv')

In [8]:
df_aug_cn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16225 entries, 0 to 16224
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     16225 non-null  int64 
 1   id             16225 non-null  object
 2   comment_text   16225 non-null  object
 3   text_cn        16225 non-null  object
 4   toxic          16225 non-null  int64 
 5   severe_toxic   16225 non-null  int64 
 6   obscene        16225 non-null  int64 
 7   threat         16225 non-null  int64 
 8   insult         16225 non-null  int64 
 9   identity_hate  16225 non-null  int64 
dtypes: int64(7), object(3)
memory usage: 1.2+ MB


In [10]:
df_aug_cn.head()

,Unnamed: 0,id,comment_text,text_cn,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,Jerk before peeing before work,1,1,1,0,1,0
1,1,0005c987bdfc9d4b,Hey... what is it..\n@ | talk .\nWhat is it......,Hey ... what is it\n@ | Talk.\nWhat is this .....,1,0,0,0,0,0
2,2,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming ...","Goodbye!\n\nDon't look, don't come back or wan...",1,0,0,0,0,0
3,3,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,Are you gay or anti-semitic?\n\nArchangel Whit...,1,0,1,0,1,1
4,4,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!","Fuck your dirty mother in the ass, fuck!",1,0,1,0,1,0
